### Fine-Tune Parameters:
    A2 and A3 appendix of Bert Paper

1. optimizer= adam
2. learning_rate=0.0001
3. beta_1=0.9
4. beta_2=0.999
5. l2 weight decay=0.01
6. warmup_steps = 10,000
7. dropout_probability = 0.1
8. activation_function = GELU
9. batch_size = 16,32
10. number_of_epocs = 2,3,4
11. learning_rate(adam) = 0.00005,0.00003,0.00002
12. max token length = 512
13. each_word_vector_dimension=768

### steps:
1. dataset preparation : balanced, cleaned
2. load model and tokenizer
3. TFTrainer, TFTrainingAruguments
4. load the model and use it

### Dataset Preparation and EDA

In [95]:
import pandas as pd
data=pd.read_csv(r"//Users/jatavathpavannaik/Documents/python/Bert/Twitter_Data.csv")

In [96]:
df=pd.DataFrame(data)

In [97]:
df["category"].value_counts()

category
 1.0    72250
 0.0    55213
-1.0    35510
Name: count, dtype: int64

In [98]:
df_negative=df[df["category"]==-1]
df_negative["category"]=2.0
df_positive=df[df["category"]==1]
df_positive=df_positive.sample(df_negative.shape[0])
df_neutral=df[df["category"]==0]
df_neutral=df_neutral.sample(df_negative.shape[0])


/var/folders/0m/9v3n4s817s5g5rw62jzf3hlr0000gn/T/ipykernel_750/3977427714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_negative["category"]=2.0


In [26]:
df=pd.concat([df_negative,df_positive,df_neutral])

In [27]:
df["category"].value_counts()

category
2.0    35510
1.0    35510
0.0    35510
Name: count, dtype: int64

In [28]:
df["clean_text"]=df["clean_text"].astype(str)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106530 entries, 0 to 111393
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  106530 non-null  object 
 1   category    106530 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.4+ MB


In [30]:
X=list(df["clean_text"])
y=list(df["category"])

#### train_test_split

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y, test_size = 0.20, random_state=0)

### tokenization

In [32]:
from transformers import DistilBertTokenizerFast
tokenizer= DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [33]:
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)

In [34]:
len(train_encoding["input_ids"])

85224

In [35]:
len(test_encoding["input_ids"])

21306

### convert tokenized inputs into tensorflow dataset

In [36]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encoding),y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encoding),y_test))

### intilizaing fine_tunning parameters

In [37]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args=TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_gpu_train_batch_size=16,
    per_gpu_eval_batch_size=32,
    eval_steps=0,
    warmup_steps=10000,
    weight_decay=0.01,
    learning_rate=0.00005,
    adam_beta1=0.9,
    adam_beta2=0.999,
)


### training the model

In [38]:
with training_args.strategy.scope(): ## for distributed computation
    model=TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=3)
trainer=TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [39]:
trainer.evaluate(test_dataset)

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


{'eval_loss': 0.08871742912956902}

In [40]:
trainer.predict(test_dataset)

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


PredictionOutput(predictions=array([[-2.5105443,  5.800942 , -2.30606  ],
       [-2.283487 ,  5.63539  , -2.3454142],
       [-1.1193345,  3.5720348, -1.8683442],
       ...,
       [-3.2836652, -1.5765861,  4.5037684],
       [-2.6568682,  5.6865735, -2.0188594],
       [-2.547587 , -2.4784482,  4.372288 ]], dtype=float32), label_ids=array([1., 1., 1., ..., 2., 1., 2.], dtype=float32), metrics={'eval_loss': 0.08876561688947247})

In [41]:
output=trainer.predict(test_dataset)[1]

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.
Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


### saving the model

In [42]:
trainer.save_model('final_model')

### metrics evaluations

In [43]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,output)

In [44]:
cm

array([[7011,    0,    0],
       [   0, 7172,    0],
       [   0,    0, 7123]])

### loading the model and using it

In [69]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)

model.load_weights('/Users/jatavathpavannaik/Documents/python/Bert/final_model/tf_model.h5')

text = "i a"
tokenized_input = tokenizer(text, return_tensors='tf')

outputs = model(tokenized_input['input_ids'])

probabilities = tf.nn.softmax(outputs.logits, axis=-1)
predicted_class = tf.argmax(probabilities, axis=-1).numpy().item()
print(f'Predicted class: {predicted_class}')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Predicted class: 2


In [48]:
predicted_class

0

### testing

In [100]:
import pandas as pd
data=pd.read_csv("/Users/jatavathpavannaik/Documents/python/Bert/Reddit_Data.csv")
data=pd.DataFrame(data)

In [103]:
data["category"].value_counts()

category
 1    15830
 0    13142
-1     8277
Name: count, dtype: int64

In [104]:
df=data.sample(500)

In [112]:
df["category"].value_counts()

category
 1    228
 0    167
-1    105
Name: count, dtype: int64

In [115]:
df.loc[df['category'] == -1, 'category'] = 2

In [74]:
text = "what the fuck"
tokenized_input = tokenizer(text, return_tensors='tf')
outputs = model(tokenized_input['input_ids'])
probabilities = tf.nn.softmax(outputs.logits, axis=-1)
predicted_class = tf.argmax(probabilities, axis=-1).numpy().item()

model_data=[]
model_data.append(predicted_class)

In [118]:
def predict_class(row):
    tokenized_input = tokenizer(str(row['clean_comment']), return_tensors='tf')
    outputs = model(tokenized_input['input_ids'])
    probabilities = tf.nn.softmax(outputs.logits, axis=-1)
    predicted_class = tf.argmax(probabilities, axis=-1).numpy().item()
    return predicted_class
model.load_weights("/Users/jatavathpavannaik/Documents/python/Bert/final_model/tf_model.h5")
# Apply the function to each row in the DataFrame and store the result in a new column 'predicted_class'
df['predicted_class'] = df.apply(predict_class, axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


In [119]:
from sklearn import metrics
Bert_classification_metrics = metrics.classification_report(df["category"], df["predicted_class"])
print("Bert_classification_metrics: \n",Bert_classification_metrics, '\n')
confusion_metrics = metrics.confusion_matrix(df["category"], df["predicted_class"])
print("confusion_metrics: \n",confusion_metrics)

Bert_classification_metrics: 
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       167
           1       0.98      0.98      0.98       228
           2       0.95      0.99      0.97       105

    accuracy                           0.98       500
   macro avg       0.97      0.98      0.98       500
weighted avg       0.98      0.98      0.98       500
 

confusion_metrics: 
 [[162   3   2]
 [  2 223   3]
 [  0   1 104]]


In [121]:
from sklearn import metrics
metrics.accuracy_score(df["category"],df["predicted_class"])

0.978

### dataset-2

In [163]:
data=pd.read_csv("/Users/jatavathpavannaik/Documents/python/Bert/training.1600000.processed.noemoticon.csv",encoding="latin-1")
data=pd.DataFrame(data)

In [166]:
data.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [168]:
data=data[["0","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]]

In [171]:
data.rename(columns={"0":"category","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":"text"},inplace=True)

/var/folders/0m/9v3n4s817s5g5rw62jzf3hlr0000gn/T/ipykernel_750/1331922322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"0":"category","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":"text"},inplace=True)


In [173]:
data["category"].value_counts()

category
4    800000
0    799999
Name: count, dtype: int64

In [175]:
data=data.sample(500)

In [178]:
data["category"].value_counts()

category
0    258
4    242
Name: count, dtype: int64

In [179]:
def predict_class(row):
    tokenized_input = tokenizer(str(row['text']), return_tensors='tf')
    outputs = model(tokenized_input['input_ids'])
    probabilities = tf.nn.softmax(outputs.logits, axis=-1)
    predicted_class = tf.argmax(probabilities, axis=-1).numpy().item()
    return predicted_class
model.load_weights("/Users/jatavathpavannaik/Documents/python/Bert/final_model/tf_model.h5")
# Apply the function to each row in the DataFrame and store the result in a new column 'predicted_class'
data['predicted_class'] = data.apply(predict_class, axis=1)

In [180]:
data

,category,text,predicted_class
1254693,4,@khali_blache Hello..you like Tom waits i see...,0
1254467,4,Congratulations to ohjoy! On 10K cache finds ...,1
1599107,4,"@ZAC_DANCE macbook, or macbook pro? either way...",1
799276,0,What're we doing?!,0
700924,0,"Ugh, I hate soldier flys. Stupid things scare...",2
...,...,...,...
245137,0,ok celeb gossip sites too,0
1065166,4,@SuziQoregon It was probably just me! It work...,2
906316,4,@Nickems that would be great!!! thanks!!,1
988517,4,I'm getting a little bit excited... I'm gettin...,1


In [182]:
data["predicted_class"].value_counts()

predicted_class
1    220
0    155
2    125
Name: count, dtype: int64

In [186]:
data.loc[data["category"]==0,'category'] = 2
data.loc[data["category"]==4,'category'] = 1

In [187]:
from sklearn import metrics
Bert_classification_metrics = metrics.classification_report(data["category"], data["predicted_class"])
print("Bert_classification_metrics: \n",Bert_classification_metrics, '\n')
confusion_metrics = metrics.confusion_matrix(data["category"], data["predicted_class"])
print("confusion_metrics: \n",confusion_metrics)

Bert_classification_metrics: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.62      0.57      0.59       242
           2       0.76      0.37      0.50       258

    accuracy                           0.46       500
   macro avg       0.46      0.31      0.36       500
weighted avg       0.69      0.46      0.54       500
 

confusion_metrics: 
 [[  0   0   0]
 [ 75 137  30]
 [ 80  83  95]]


/Users/jatavathpavannaik/anaconda3/envs/Bert/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jatavathpavannaik/anaconda3/envs/Bert/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jatavathpavannaik/anaconda3/envs/Bert/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [191]:
df = data[(data["predicted_class"] == 1) | (data["predicted_class"] == 2)]


In [199]:
df["text"][291164]

"@rioblaze awww... thanks...Even thought I can't figure out to put a picture up hurr.. "

In [192]:
from sklearn import metrics
metrics.accuracy_score(df["category"],df["predicted_class"])

0.672463768115942

### airline sentiment

In [1]:
import pandas as pd
data=pd.read_csv("/Users/jatavathpavannaik/Documents/python/Bert/Tweets.csv")
df=pd.DataFrame(data)

In [3]:
df=df[["airline_sentiment","text"]]

In [7]:
df=df.sample(500)

In [9]:
df["airline_sentiment"].value_counts()

airline_sentiment
negative    317
neutral      99
positive     84
Name: count, dtype: int64

In [11]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)

model.load_weights('/Users/jatavathpavannaik/Documents/python/Bert/final_model/tf_model.h5')

/Users/jatavathpavannaik/anaconda3/envs/Bert/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-06 12:27:15.426538: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-01-06 12:27:15.426563: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-06 12:27:15.426571: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-06 12:27:15.426915: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-06 12:27:15.427373: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with

In [12]:
def predict_class(row):
    tokenized_input = tokenizer(str(row['text']), return_tensors='tf')
    outputs = model(tokenized_input['input_ids'])
    probabilities = tf.nn.softmax(outputs.logits, axis=-1)
    predicted_class = tf.argmax(probabilities, axis=-1).numpy().item()
    return predicted_class
model.load_weights("/Users/jatavathpavannaik/Documents/python/Bert/final_model/tf_model.h5")
# Apply the function to each row in the DataFrame and store the result in a new column 'predicted_class'
df['predicted_class'] = df.apply(predict_class, axis=1)

In [17]:
df.loc[df["airline_sentiment"]=="negative", "airline_sentiment"]=2
df.loc[df["airline_sentiment"]=="positive", "airline_sentiment"]=1
df.loc[df["airline_sentiment"]=="neutral", "airline_sentiment"]=0

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 7538 to 10758
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  500 non-null    object
 1   text               500 non-null    object
 2   predicted_class    500 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 15.6+ KB


In [21]:
from sklearn import metrics
Bert_classification_metrics = metrics.classification_report(df["airline_sentiment"].astype(int), df["predicted_class"])
print("Bert_classification_metrics: \n",Bert_classification_metrics, '\n')
confusion_metrics = metrics.confusion_matrix(df["airline_sentiment"].astype(int), df["predicted_class"])
print("confusion_metrics: \n",confusion_metrics)

Bert_classification_metrics: 
               precision    recall  f1-score   support

           0       0.31      0.55      0.39        99
           1       0.36      0.74      0.49        84
           2       0.90      0.44      0.59       317

    accuracy                           0.51       500
   macro avg       0.52      0.57      0.49       500
weighted avg       0.69      0.51      0.53       500
 

confusion_metrics: 
 [[ 54  34  11]
 [ 17  62   5]
 [105  74 138]]


In [22]:
from sklearn import metrics
metrics.accuracy_score(df["airline_sentiment"].astype(int), df["predicted_class"])

0.508

In [24]:
df.to_excel("test.xlsx")

### comparision of distilbert base and our model

In [25]:
import pandas as pd
data=pd.read_csv("/Users/jatavathpavannaik/Documents/python/Bert/Tweets.csv")
df=pd.DataFrame(data)
df=df[["airline_sentiment","text"]]

In [32]:
df=df.loc[(df["airline_sentiment"]=="negative") | (df["airline_sentiment"]=="positive") ]
df=df.sample(500)

In [34]:
df["airline_sentiment"].value_counts()

airline_sentiment
negative    402
positive     98
Name: count, dtype: int64

In [75]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)

model.load_weights('/Users/jatavathpavannaik/Documents/python/Bert/final_model/tf_model.h5')

text = "The photo is of today when my dog ​​was unable to urinate, then we went to  BVC college & hospital at  am, then the guard said that the hospital is closed today, so I asked if emergency is also closed, then said yes everything is closed? yadavtejashwi ji plz speak."
tokenized_input = tokenizer(text, return_tensors='tf')

outputs = model(tokenized_input['input_ids'])

probabilities = tf.nn.softmax(outputs.logits, axis=-1)
predicted_class = tf.argmax(probabilities, axis=-1).numpy().item()
print(f'Predicted class: {predicted_class}')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Predicted class: 2


In [76]:
probabilities

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.00988188, 0.01722025, 0.97289795]], dtype=float32)>

In [31]:
from transformers import pipeline
import pandas as pd
sentiment_pipeline = pipeline("sentiment-analysis","distilbert-base-uncased-finetuned-sst-2-english")

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [32]:
sentiment_pipeline("congratulations")

[{'label': 'POSITIVE', 'score': 0.9998007416725159}]

In [37]:
sentiment_pipeline("")

[{'label': 'POSITIVE', 'score': 0.9994704127311707}]

In [28]:
from transformers import pipeline
import pandas as pd
sentiment_pipeline = pipeline("sentiment-analysis","distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [66]:
sentiment_pipeline("Possibly rate per job, as cut money, has been fixed.")

[{'label': 'NEGATIVE', 'score': 0.9985482096672058}]

In [30]:
sentiment_pipeline("congratulations")

[{'label': 'LABEL_1', 'score': 0.5153689384460449}]

### on own dataset

In [77]:
import pandas as pd
data=pd.read_excel("/Users/jatavathpavannaik/Documents/python/Bert/Data_Set.xlsx",sheet_name="test")
df=pd.DataFrame(data)

In [80]:
df

,Text,len_of_text
0,"Why should she be referred to as ""My Lord""?",43
1,: and she did that I have also voted for her,44
2,Call him Chenal Ghosh which suits that Joker,44
3,Knowledge is power. thank you. God bless you.,45
4,she learnt how to feed them with our Tax money,47
...,...,...
494,One more. Fact here the cases are of rd mutati...,64
495,I agree with you. BJP hatao Bharat bachao RSS ...,64
496,It is very unfortunately matter which was happ...,65
497,Time will tell who was right and acted as per ...,65


In [81]:
def predict_class(row):
    tokenized_input = tokenizer(str(row['Text']), return_tensors='tf')
    outputs = model(tokenized_input['input_ids'])
    probabilities = tf.nn.softmax(outputs.logits, axis=-1)
    predicted_class = tf.argmax(probabilities, axis=-1).numpy().item()
    return predicted_class
model.load_weights("/Users/jatavathpavannaik/Documents/python/Bert/final_model/tf_model.h5")
# Apply the function to each row in the DataFrame and store the result in a new column 'predicted_class'
df['predicted_class'] = df.apply(predict_class, axis=1)

In [82]:
df.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
